### Authenticate and configure API client
- set environment variable for insecure transport
- import api client and auth code response
- authenticate and configure api client
- set default header for api client (to include bearer token) - this is required for all api calls
  - setting the default header will soon be handled by the auth_utils_requests_oathlib.login_interactive method

In [ ]:
%env OAUTHLIB_INSECURE_TRANSPORT=1;

from vcell_client import ApiClient, AuthCodeResponse, Configuration, Publication
from vcell_client.auth import auth_utils_requests_oathlib as vc_auth

api_url: str = "http://localhost:9000"
client_id: str = 'backend-service'
authorization_url: str = 'http://localhost:60239/realms/quarkus/protocol/openid-connect/auth'

auth_code_response: AuthCodeResponse = vc_auth.login_interactive(api_base_url=api_url, client_id=client_id,
                                                                 authorization_base_url=authorization_url)
api_client = ApiClient(configuration=Configuration(host=api_url, access_token=auth_code_response.access_token))
api_client.set_default_header('Authorization', f'Bearer {auth_code_response.access_token}')

### Test Publication API
- get publications
- add publication
- delete publication
- get publications

In [ ]:
from vcell_client.api.publication_resource_api import PublicationResourceApi
from datetime import date

publication_api = PublicationResourceApi(api_client)
pubs: list[Publication] = publication_api.get_publications()
display(f"initial publications: {pubs}")

pub1 = Publication(title="publication 1", authors=["author 1", "author 2"], year=1995, citation="citation 1",
    pubmedid="pubmedid 1", doi="doi 1", endnoteid=1, url="url 1", wittid=1, biomodel_refs=[], mathmodel_refs=[], var_date=date.today())
pub2 = pub1.model_copy()
pub2.title="publication 2"

pub1_key: int = publication_api.add_publication(publication=pub1)
pub2_key: int = publication_api.add_publication(publication=pub2)

more_pubs: list[Publication] = publication_api.get_publications()
display(f"more publications: {more_pubs}")

pub1_from_db = publication_api.get_publication(id=pub1_key)
display(f"publication 1 from db: {pub1_from_db}")

for pub in more_pubs:
    publication_api.delete_publication(id=pub.pub_key)

final_pubs: list[Publication] = publication_api.get_publications()
display(f"final publications: {final_pubs}")

